# Lesson 3: Reflection and Blogpost Writing

## Setup

In [1]:
llm_config = {"model": "gpt-3.5-turbo"}

## The task!

In [2]:
task = '''
        Write a concise but engaging blogpost about
       DeepLearning.AI. Make sure the blogpost is
       within 100 words.
       '''


## Create a writer agent

In [3]:
import autogen

writer = autogen.AssistantAgent(
    name="Writer",
    system_message="You are a writer. You write engaging and concise " 
        "blogpost (with title) on given topics. You must polish your "
        "writing based on the feedback you receive and give a refined "
        "version. Only return your final work without additional comments.",
    llm_config=llm_config,
)

In [4]:
reply = writer.generate_reply(messages=[{"content": task, "role": "user"}])

In [5]:
print(reply)

Title: Unveiling the Power of DeepLearning.AI

Delve into the world of artificial intelligence with DeepLearning.AI! Spearheaded by industry leader Andrew Ng, this online platform offers a gateway to mastering deep learning. From neural networks to natural language processing, it equips learners with in-demand AI skills. The interactive courses and hands-on projects make complex concepts easily digestible for beginners and experts alike. Whether you're a tech enthusiast or a professional looking to upskill, DeepLearning.AI provides a dynamic learning experience that can propel your career forward in this rapidly evolving field. Embrace the future of AI today!


## Adding reflection 

Create a critic agent to reflect on the work of the writer agent.

In [6]:
critic = autogen.AssistantAgent(
    name="Critic",
    is_termination_msg=lambda x: x.get("content", "").find("TERMINATE") >= 0,
    llm_config=llm_config,
    system_message="You are a critic. You review the work of "
                "the writer and provide constructive "
                "feedback to help improve the quality of the content.",
)

In [7]:
res = critic.initiate_chat(
    recipient=writer,
    message=task,
    max_turns=2,
    summary_method="last_msg"
)

Critic (to Writer):


        Write a concise but engaging blogpost about
       DeepLearning.AI. Make sure the blogpost is
       within 100 words.
       

--------------------------------------------------------------------------------
Writer (to Critic):

Title: Unveiling the Power of DeepLearning.AI

Delve into the world of artificial intelligence with DeepLearning.AI! Spearheaded by industry leader Andrew Ng, this online platform offers a gateway to mastering deep learning. From neural networks to natural language processing, it equips learners with in-demand AI skills. The interactive courses and hands-on projects make complex concepts easily digestible for beginners and experts alike. Whether you're a tech enthusiast or a professional looking to upskill, DeepLearning.AI provides a dynamic learning experience that can propel your career forward in this rapidly evolving field. Embrace the future of AI today!

-----------------------------------------------------------------------

## Nested chat

In [8]:
SEO_reviewer = autogen.AssistantAgent(
    name="SEO Reviewer",
    llm_config=llm_config,
    system_message="You are an SEO reviewer, known for "
        "your ability to optimize content for search engines, "
        "ensuring that it ranks well and attracts organic traffic. " 
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role.",
)


In [9]:
legal_reviewer = autogen.AssistantAgent(
    name="Legal Reviewer",
    llm_config=llm_config,
    system_message="You are a legal reviewer, known for "
        "your ability to ensure that content is legally compliant "
        "and free from any potential legal issues. "
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role.",
)

In [10]:
ethics_reviewer = autogen.AssistantAgent(
    name="Ethics Reviewer",
    llm_config=llm_config,
    system_message="You are an ethics reviewer, known for "
        "your ability to ensure that content is ethically sound "
        "and free from any potential ethical issues. " 
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role. ",
)

In [11]:
meta_reviewer = autogen.AssistantAgent(
    name="Meta Reviewer",
    llm_config=llm_config,
    system_message="You are a meta reviewer, you aggragate and review "
    "the work of other reviewers and give a final suggestion on the content.",
)

## Orchestrate the nested chats to solve the task

In [19]:
def reflection_message(recipient, messages, sender, config):
#     print(recipient.chat_messages_for_summary(sender)[-1])
    return f'''Review the following content. 
            \n\n {recipient.chat_messages_for_summary(sender)[-1]['content']}'''

review_chats = [
    {
     "recipient": SEO_reviewer, 
     "message": reflection_message, 
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" : 
        "Return review into as JSON object only:"
        "{'Reviewer': '', 'Review': ''}. Here Reviewer should be your role",},
     "max_turns": 1},
    {
    "recipient": legal_reviewer, "message": reflection_message, 
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" : 
        "Return review into as JSON object only:"
        "{'Reviewer': '', 'Review': ''}.",},
     "max_turns": 1},
    {"recipient": ethics_reviewer, "message": reflection_message, 
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" : 
        "Return review into as JSON object only:"
        "{'reviewer': '', 'review': ''}",},
     "max_turns": 1},
     {"recipient": meta_reviewer, 
      "message": "Aggregrate feedback from all reviewers and give final suggestions on the writing.", 
     "max_turns": 1},
]


In [20]:
critic.register_nested_chats(
    review_chats,
    trigger=writer,
)

**Note**: You might get a slightly different response than what's shown in the video. Feel free to try different task.

In [21]:
res = critic.initiate_chat(
    recipient=writer,
    message=task,
    max_turns=2,
    summary_method="last_msg"
)

Critic (to Writer):


        Write a concise but engaging blogpost about
       DeepLearning.AI. Make sure the blogpost is
       within 100 words.
       

--------------------------------------------------------------------------------
Writer (to Critic):

Title: Unveiling the Power of DeepLearning.AI

Delve into the world of artificial intelligence with DeepLearning.AI! Spearheaded by industry leader Andrew Ng, this online platform offers a gateway to mastering deep learning. From neural networks to natural language processing, it equips learners with in-demand AI skills. The interactive courses and hands-on projects make complex concepts easily digestible for beginners and experts alike. Whether you're a tech enthusiast or a professional looking to upskill, DeepLearning.AI provides a dynamic learning experience that can propel your career forward in this rapidly evolving field. Embrace the future of AI today!

-----------------------------------------------------------------------


--------------------------------------------------------------------------------
Critic (to Writer):

Aggregated feedback:
1. SEO Analyst: Focus on optimizing the title with relevant keywords, implement meta tags, and internal linking strategies for better search visibility.
2. Legal Reviewer: Ensure compliance with intellectual property rights, avoid unsubstantiated claims, and verify accuracy in representations.
3. Ethics Reviewer: Back up claims about effectiveness with verifiable data, avoid creating unrealistic expectations with promotional language, and ensure accurate representation of platform benefits without overstating capabilities.

Final suggestion on the writing:
1. Optimize the title with relevant keywords for improved search visibility.
2. Ensure compliance with intellectual property rights and avoid making unsubstantiated claims to maintain credibility.
3. Back up claims with verifiable data to avoid misleading consumers and steer clear of creating unrealistic expecta

## Get the summary

In [15]:
print(res.summary)

Title: Mastering AI with DeepLearning.AI

Dive deep into the realm of artificial intelligence with DeepLearning.AI, led by renowned expert Andrew Ng. The platform offers a comprehensive range of courses, from neural networks to natural language processing, enabling learners to hone essential AI skills. With interactive modules and practical projects, DeepLearning.AI caters to beginners and seasoned professionals alike. Whether you're an enthusiast or a career-driven individual, this dynamic learning experience can boost your expertise in this rapidly advancing field. Embrace the future of AI today with DeepLearning.AI!
